# プログラミング未経験者のためのデータ解析・機械学習
# 第 12 章　化学構造の扱い

## Jupyter Notebook の有用なショートカットのまとめ
- <kbd>Esc</kbd>: コマンドモードに移行（セルの枠が青）
- <kbd>Enter</kbd>: 編集モードに移行（セルの枠が緑）
- コマンドモードで <kbd>M</kbd>: Markdown セル (説明・メモを書く用) に変更
- コマンドモードで <kbd>Y</kbd>: Code セル (Python コードを書く用) に変更
- コマンドモードで <kbd>H</kbd>: ヘルプを表示
- コマンドモードで <kbd>A</kbd>: ひとつ**上**に空のセルを挿入
- コマンドモードで <kbd>B</kbd>: ひとつ**下**に空のセルを挿入
- コマンドモードで <kbd>D</kbd><kbd>D</kbd>: セルを削除
- <kbd>Ctrl</kbd>+<kbd>Enter</kbd>: セルの内容を実行
- <kbd>Shift</kbd>+<kbd>Enter</kbd>: セルの内容を実行して下へ

わからないことがありましたら、関係する単語やエラーの文章などでウェブ検索してご自身で調べてみましょう。

下のセルを実行して、RDKit version: 2020.03.2 と表示されれば (バージョンは変わるかもしれません)、RDKit のインストールは成功です。「RDKit WARNING: …」というメッセージが出た場合も問題なくインストールは成功しています。

In [ ]:
from rdkit import rdBase
print('RDKit version: {0}'.format(rdBase.rdkitVersion))

## 化学構造の表現方法

### SMILES

In [ ]:
from rdkit import Chem

分子の SMILES を直接書くことは普通はなく、基本的にはウェブサービスや構造式描画ソフトウェアで描画して、それを SMILES に変換することで SMILES を作成しますが、今回は簡単な化学構造を扱い SMILES を直接書いて、RDKit で扱える形式に変換します。

In [ ]:
alanine_from_smiles = Chem.MolFromSmiles('CC(N)C(=O)O') # アラニンの SMILES を読み込みます

In [ ]:
type(alanine_from_smiles) # rdkit.Chem.rdchem.Mol と表示されます。Mol 型の変数です。

Mol 型の変数を直接扱うことはなく RDKit の関数を介して扱います。

In [ ]:
Chem.MolToSmiles(alanine_from_smiles) # Mol 型の変数から SMILES の文字列に変換できます

分子の描画

In [ ]:
from rdkit.Chem import Draw

In [ ]:
Draw.MolToImage(alanine_from_smiles)

水素原子が自動的に付加されています。

#### 【練習】
エタノール、ベンゼンについて、それぞれテキストにある SMILES を用いて分子として読み込み、描画してみましょう。コード例は一番下にあります。

### MOL file

In [ ]:
alanine_from_molfile = Chem.MolFromMolFile('alanine.mol') # アラニンの MOL file を読み込みます

In [ ]:
type(alanine_from_molfile) # rdkit.Chem.rdchem.Mol と表示されます。Mol 型の変数です

In [ ]:
Chem.MolToSmiles(alanine_from_molfile) # Mol 型の変数から SMILES の文字列に変換できます

分子の描画

In [ ]:
Chem.Draw.MolToImage(alanine_from_molfile)

水素原子が自動的に付加されています。

#### 【練習】
エタノール、ベンゼンについて、MOL file を分子として読み込み、描画してみましょう。それぞれの MOL file は ethanol.mol、 benzene.mol です。コード例は一番下にあります。

## 化合物群の扱い

SMILES に基づくデータベース

#### SMILES付き沸点のデータセット (molecules_with_boiling_point.csv)
Hall and Story が収集した[沸点のデータセット](https://pubs.acs.org/doi/abs/10.1021/ci960375x)。294 個の化合物について、SMILES と測定された沸点 (Boiling Point) が格納された csv ファイルです。

In [ ]:
import pandas as pd

In [ ]:
dataset = pd.read_csv('molecules_with_boiling_point.csv', index_col=0) # SMILES付き沸点のデータセットの読み込み

In [ ]:
dataset # 念のため確認

In [ ]:
y = dataset.iloc[:, 1]

In [ ]:
y

リスト型の変数 `molecules_from_smiles` に、SMILES から変換した分子を追加していきます。

In [ ]:
molecules_from_smiles = [] # 空のリスト型の変数
for smiles in dataset.iloc[:, 0]:
    molecules_from_smiles.append(Chem.MolFromSmiles(smiles))

In [ ]:
len(molecules_from_smiles)

MOL file に基づくデータベース

#### 沸点のデータセットの sdf ファイル (molecules_with_boiling_point.sdf)
Hall and Story が収集した[沸点のデータセット](https://pubs.acs.org/doi/abs/10.1021/ci960375x)。294 個の化合物の化学構造情報と沸点 (Boiling Point) が一緒に格納された sdf ファイルです。

In [ ]:
sdf = Chem.SDMolSupplier('molecules_with_boiling_point.sdf')

化合物ごとに、リスト型の変数 `y_from_sdf` に物性値を、リスト型の変数 `molecules_from_sdf` に Mol 型の化学構造情報を追加していきます。

In [ ]:
y_from_sdf = []
molecules_from_sdf = []
for molecule in sdf:
    y_from_sdf.append(float(molecule.GetProp('BoilingPoint')))
    molecules_from_sdf.append(molecule)

In [ ]:
len(molecules_from_sdf) # 念のため確認

In [ ]:
y_from_sdf = pd.DataFrame(y_from_sdf) # DataFrame 型に変換

In [ ]:
y_from_sdf.columns = ['BoilingPoint']

In [ ]:
y_from_sdf # 念のため確認

## 化学構造の数値化

In [ ]:
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors

In [ ]:
Descriptors.descList

記述子の名前を取得します。リスト型の変数 `descriptor_names` に記述子の名前を追加していきます。

In [ ]:
descriptor_names = []
for descriptor_information in Descriptors.descList:
    descriptor_names.append(descriptor_information[0])

In [ ]:
descriptor_names # 念のため確認

In [ ]:
len(descriptor_names)

In [ ]:
descriptor_calculator = MoleculeDescriptors.MolecularDescriptorCalculator(descriptor_names)

記述子を計算します。計算終了まで少し時間がかかります。

In [ ]:
descriptors = []
smiles = []
for molecule in molecules_from_smiles:
    descriptors.append(descriptor_calculator.CalcDescriptors(molecule))
    smiles.append(Chem.MolToSmiles(molecule))

In [ ]:
descriptors = pd.DataFrame(descriptors) # DataFrame 型に変換

In [ ]:
descriptors.columns = descriptor_names # 列名を記述子名に

In [ ]:
descriptors.index = smiles # 行名を SMILES に

In [ ]:
descriptors # 念のため確認

In [ ]:
descriptors.to_csv('descriptors.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

`descriptors` をこれまでの連載における `x` とすることで、化学構造のデータセットでもデータの可視化・クラスタリング・クラス分類・回帰分析などができるようになります。

## 化学構造のデータセットを扱うときの注意点およびデータセットの前処理

データセットのトレーニングデータとテストデータへの分割

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# ランダムにトレーニングデータとテストデータとに分割。random_state に数字を与えることで、別のときに同じ数字を使えば、ランダムとはいえ同じ結果にすることができます
x_train, x_test, y_train, y_test = train_test_split(descriptors, y, test_size=94, shuffle=True, random_state=99)

In [ ]:
x_train.std()

標準偏差が 0 の記述子があることを確認しましょう。標準偏差が 0 では、変数の標準化 (オートスケーリング) のときに 0 で割ることになってしまいます。標準偏差が 0 の記述子を削除します。

In [ ]:
x_train.std() == 0 # 標準偏差が 0 の記述子だけ True

In [ ]:
x_train.columns[x_train.std() == 0] # 標準偏差が 0 の記述子

In [ ]:
len(x_train.columns[x_train.std() == 0]) # 標準偏差が 0 の記述子の数

In [ ]:
x_train_new = x_train.drop(x_train.columns[x_train.std() == 0], axis=1) # トレーニングデータから標準偏差が 0 の記述子を削除

In [ ]:
x_train_new.shape # 大きさを確認

In [ ]:
x_test_new = x_test.drop(x_train.columns[x_train.std() == 0], axis=1) # テストデータから標準偏差が 0 の記述子を削除

In [ ]:
x_test_new.shape # 大きさを確認

記述子の標準化 (オートスケーリング)

In [ ]:
autoscaled_x_train = (x_train_new - x_train_new.mean()) / x_train_new.std() # トレーニングデータの説明変数の標準化。x_train の代わりに x_train_new を用います

In [ ]:
autoscaled_x_test = (x_test_new - x_train_new.mean()) / x_train_new.std() # テストデータの説明変数の標準化。x_train の代わりに x_train_new を、x_test の代わりに x_test_new を用います

In [ ]:
autoscaled_y_train = (y_train - y_train.mean()) / y_train.std() # トレーニングデータの目的変数の標準化

#### 【練習】
molecules_with_logS.csv, molecules_with_logS.sdf それぞれについて、データセットを読み込み、化学構造に対して記述子を計算しましょう。さらに、サンプル全体の 3 割をテストデータにしたときに、トレーニングデータにおいて標準偏差が 0 になる記述子を、トレーニングデータ・テストデータそれぞれで削除して、オートスケーリングしましょう。コード例は一番下にあります。

#### SMILES付き水溶解度のデータセット (molecules_with_logS.csv)
Hou et al. が収集した[水溶解度のデータセット](https://pubs.acs.org/doi/abs/10.1021/ci034184n)。1290 個の化合物について、SMILESと水溶解度の対数値 logS が格納された csv ファイルです。

#### 水溶解度のデータセットの sdf ファイル (molecules_with_logS.sdf)
Hou et al. が収集した[水溶解度のデータセット](https://pubs.acs.org/doi/abs/10.1021/ci034184n)。1290 個の化合物の化学構造情報と水溶解度の対数値 logS が一緒に格納された sdf ファイルです。

#### 【コード例】
エタノール、ベンゼンについて、それぞれテキストにある SMILES を用いて分子として読み込み、描画してみましょう。

In [ ]:
ethanol_from_smiles = Chem.MolFromSmiles('CCO') # エタノール

In [ ]:
Draw.MolToImage(ethanol_from_smiles)

In [ ]:
benzene_from_smiles = Chem.MolFromSmiles('c1ccccc1') # ベンゼン

In [ ]:
Draw.MolToImage(benzene_from_smiles)

#### 【コード例】
エタノール、ベンゼンについて、mol ファイルを分子として読み込み、描画してみましょう。それぞれの MOL file は ethanol.mol、 benzene.mol です。

In [ ]:
ethanol_from_molfile = Chem.MolFromMolFile('ethanol.mol') # エタノール

In [ ]:
Draw.MolToImage(ethanol_from_molfile)

In [ ]:
benzene_from_molfile = Chem.MolFromMolFile('benzene.mol') # ベンゼン

In [ ]:
Draw.MolToImage(benzene_from_molfile)

#### 【コード例】
molecules_with_logS_smiles.csv, molecules_with_logS.sdf それぞれについて、データセットを読み込み、化学構造に対して記述子を計算しましょう。さらに、サンプル全体の 3 割をテストデータにしたときに、トレーニングデータにおいて標準偏差が 0 になる記述子を、トレーニングデータ・テストデータそれぞれで削除して、オートスケーリングしましょう。

SMILES に基づくデータベースの場合

In [ ]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors

In [ ]:
dataset = pd.read_csv('molecules_with_logS.csv', index_col=0) # SMILES付き水溶解度のデータセットの読み込み

In [ ]:
dataset # 念のため確認

In [ ]:
y = dataset.iloc[:, 1]

In [ ]:
y

リスト型の変数 `molecules_from_smiles` に SMILES から変換した分子を追加していきます。

In [ ]:
molecules_from_smiles = [] # 空のリスト型の変数
for smiles in dataset.iloc[:, 0]:
    molecules_from_smiles.append(Chem.MolFromSmiles(smiles))

In [ ]:
len(molecules_from_smiles)

記述子の名前を取得します。リスト型の変数 `descriptor_names` に記述子の名前を追加していきます。

In [ ]:
descriptor_names = []
for descriptor_information in Descriptors.descList:
    descriptor_names.append(descriptor_information[0])

In [ ]:
descriptor_names # 念のため確認

In [ ]:
len(descriptor_names)

記述子を計算します。

In [ ]:
descriptor_calculator = MoleculeDescriptors.MolecularDescriptorCalculator(descriptor_names)

In [ ]:
descriptors = []
smiles = []
for molecule in molecules_from_smiles:
    descriptors.append(descriptor_calculator.CalcDescriptors(molecule))
    smiles.append(Chem.MolToSmiles(molecule))

In [ ]:
descriptors = pd.DataFrame(descriptors) # DataFrame 型に変換

In [ ]:
descriptors.columns = descriptor_names # 列名を記述子名に

In [ ]:
descriptors.index = smiles # 行名を SMILES に

In [ ]:
descriptors # 念のため確認

In [ ]:
descriptors.to_csv('descriptors.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

`descriptors` をこれまでの連載における `x` とすることで、化学構造のデータセットでもデータの可視化・クラスタリング・クラス分類・回帰分析などができるようになります。

データセットのトレーニングデータとテストデータへの分割

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# ランダムにトレーニングデータとテストデータとに分割。random_state に数字を与えることで、別のときに同じ数字を使えば、ランダムとはいえ同じ結果にすることができます
x_train, x_test, y_train, y_test = train_test_split(descriptors, y, test_size=387, shuffle=True, random_state=99)

In [ ]:
x_train.std() == 0 # 標準偏差が 0 の記述子だけ True

In [ ]:
x_train.columns[x_train.std() == 0] # 標準偏差が 0 の記述子

In [ ]:
len(x_train.columns[x_train.std() == 0]) # 標準偏差が 0 の記述子の数

In [ ]:
x_train_new = x_train.drop(x_train.columns[x_train.std() == 0], axis=1) # トレーニングデータから標準偏差が 0 の記述子を削除

In [ ]:
x_train_new.shape # 大きさを確認

In [ ]:
x_test_new = x_test.drop(x_train.columns[x_train.std() == 0], axis=1) # テストデータから標準偏差が 0 の記述子を削除

In [ ]:
x_test_new.shape # 大きさを確認

記述子の標準化 (オートスケーリング)

In [ ]:
autoscaled_x_train = (x_train_new - x_train_new.mean()) / x_train_new.std() # トレーニングデータの説明変数の標準化。x_train の代わりに x_train_new を用います

In [ ]:
autoscaled_x_test = (x_test_new - x_train_new.mean()) / x_train_new.std() # テストデータの説明変数の標準化。x_train の代わりに x_train_new を、x_test の代わりに x_test_new を用います

In [ ]:
autoscaled_y_train = (y_train - y_train.mean()) / y_train.std() # トレーニングデータの目的変数の標準化

SDF の場合

In [ ]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors

In [ ]:
sdf = Chem.SDMolSupplier('molecules_with_logS.sdf')

化合物ごとに、リスト型の変数 `y` に物性値を、リスト型の変数 `molecules_from_sdf` に Mol 型の化学構造情報を追加していきます。

In [ ]:
y = []
molecules_from_sdf = []
for molecule in sdf:
    # y.append(molecule.GetProp('logS'))
    y.append(float(molecule.GetProp('logS')))
    molecules_from_sdf.append(molecule)

In [ ]:
len(molecules_from_sdf) # 念のため確認

In [ ]:
y = pd.DataFrame(y) # DataFrame 型に変換

In [ ]:
y.columns = ['logS']

In [ ]:
y # 念のため確認

記述子の名前を取得します。リスト型の変数 `descriptor_names` に記述子の名前を追加していきます。

In [ ]:
descriptor_names = []
for descriptor_information in Descriptors.descList:
    descriptor_names.append(descriptor_information[0])

In [ ]:
descriptor_names # 念のため確認

In [ ]:
len(descriptor_names)

記述子を計算します。

In [ ]:
descriptor_calculator = MoleculeDescriptors.MolecularDescriptorCalculator(descriptor_names)

In [ ]:
descriptors = []
smiles = []
for molecule in molecules_from_sdf:
    descriptors.append(descriptor_calculator.CalcDescriptors(molecule))
    smiles.append(Chem.MolToSmiles(molecule))

In [ ]:
descriptors = pd.DataFrame(descriptors) # DataFrame 型に変換

In [ ]:
descriptors.columns = descriptor_names # 列名を記述子名に

In [ ]:
descriptors.index = smiles # 行名を SMILES に

In [ ]:
descriptors # 念のため確認

In [ ]:
descriptors.to_csv('descriptors.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

`descriptors` をこれまでの連載における `x` とすることで、化学構造のデータセットでもデータの可視化・クラスタリング・クラス分類・回帰分析などができるようになります。

データセットのトレーニングデータとテストデータへの分割

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# ランダムにトレーニングデータとテストデータとに分割。random_state に数字を与えることで、別のときに同じ数字を使えば、ランダムとはいえ同じ結果にすることができます
x_train, x_test, y_train, y_test = train_test_split(descriptors, y, test_size=387, shuffle=True, random_state=99)

In [ ]:
x_train.std() == 0 # 標準偏差が 0 の記述子だけ True

In [ ]:
x_train.columns[x_train.std() == 0] # 標準偏差が 0 の記述子

In [ ]:
len(x_train.columns[x_train.std() == 0]) # 標準偏差が 0 の記述子の数

In [ ]:
x_train_new = x_train.drop(x_train.columns[x_train.std() == 0], axis=1) # トレーニングデータから標準偏差が 0 の記述子を削除

In [ ]:
x_train_new.shape # 大きさを確認

In [ ]:
x_test_new = x_test.drop(x_train.columns[x_train.std() == 0], axis=1) # テストデータから標準偏差が 0 の記述子を削除

In [ ]:
x_test_new.shape # 大きさを確認

記述子の標準化 (オートスケーリング)

In [ ]:
autoscaled_x_train = (x_train_new - x_train_new.mean()) / x_train_new.std() # トレーニングデータの説明変数の標準化。x_train の代わりに x_train_new を用います

In [ ]:
autoscaled_x_test = (x_test_new - x_train_new.mean()) / x_train_new.std() # テストデータの説明変数の標準化。x_train の代わりに x_train_new を、x_test の代わりに x_test_new を用います

In [ ]:
autoscaled_y_train = (y_train - y_train.mean()) / y_train.std() # トレーニングデータの目的変数の標準化